<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Train_polyphonic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [5]:
%%capture
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision.transforms.functional as TF 

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os
import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
from torch import optim
from torch.optim import lr_scheduler
from functools import partial
import matplotlib.pyplot as plt
#!pip install partitura
! pip install git+https://github.com/CPJKU/partitura.git@develop
import partitura
import statistics



!git clone https://github.com/aim56009/AI-MA_project.git

#Dataloader - Set the dataset

In [1]:
PATH_TO_DATA = "AI-MA_project/pr_polyphonic"

batch_size = 1 
workers = 0

In [ ]:
class MusicDataset_polyphonic(Dataset):

    def __init__(self, data_dir, transforms=None):
        self.transforms = transforms
        self.data_dir = data_dir

        self.name_list = ["1f01","1f02","1f03","1f04","1f05","1f06","1f07","1f08","1f09","1f10","1f11","1f12","1f13","1f14","1f15","1f16","1f17","1f18","1f19","1f20","1f21","1f22","1f23","1f24","2f01","2f02","2f03","2f04","2f05","2f06","2f07","2f08","2f09","2f10","2f11","2f12","2f13","2f14","2f15","2f16","2f17","2f18","2f19","2f20","2f21","2f22","2f23","2f24"]
        self.name_list_voice_3 =  ['1f01', '1f05', '1f12', '1f14', '1f16', '1f17', '1f18', '1f23', '1f24', '2f02', '2f05', '2f07', '2f08', '2f09', '2f16', '2f17',  '2f22', '2f23']
        labels = ["voice_0", "voice_1", "voice_2", "voice_3", "voice_all"]
        self.labels = labels
        self.pr_dict = {}
        len_list = []
        nbr_voices_list = []
        file_names_list = []

        for iLabel in range(len(labels)):
            
            if iLabel == 4:   
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl")))   
                for name in file_names:
                    with open(name ,'rb') as f: ### normal sollte es egal sein wenn voice_4 bei manchen nicht existiert - wenn nicht condition einführen damit das funktioniert
                        loaded_obj = pickle.load(f)  
                        voice_files.append(loaded_obj) 
                        len_list.append(len(loaded_obj.T))

                        file_names_list.append(name[-8:-4])

                    if "AI-MA_project/bach_pr_fugues/voice_3/voice_3_" + name[49:53] + ".pkl" in sorted(glob.glob(os.path.join(PATH_TO_DATA, "voice_3", "*.pkl"))):
                        nbr_voices_list.append(4)
                    else:
                        nbr_voices_list.append(3)
                        
                self.pr_dict[self.labels[iLabel]] = voice_files 
                self.pr_dict["length"] = len_list
                self.pr_dict["nbr_voices"] = nbr_voices_list
                self.pr_dict["name"] = file_names_list
                #print(self.pr_dict["name"])


            if iLabel == 3:  
                voice_files = []
                file_names_3 = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl"))) 
                file_names_2 = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[2], "*.pkl")))   
                
                ###### loop over all filnames in voices_2 and if an element there is not present in voices_3: append "missing" to the voice_files of label=3 => important bc. self.pr_dict[voice_3] has then len 42 and otherwise it would only have len 18  .. these "missing" el are not considered later in the dataloader (if len=3 is a diff case of get_idx)
                for name in file_names_2:
                    if name[45:49] in self.name_list_voice_3:
                      correct_name_3 = "AI-MA_project/bach_pr_fugues/voice_3/voice_3_" + name[45:49] + ".pkl"
                      with open(correct_name_3 ,'rb') as f:  
                            loaded_obj = pickle.load(f)  
                            voice_files.append(loaded_obj)
                    else:
                      voice_files.append("missing")

                self.pr_dict[self.labels[iLabel]] = voice_files 
                

                
            else:
                voice_files = []
                file_names = sorted(glob.glob(os.path.join(PATH_TO_DATA, self.labels[iLabel], "*.pkl"))) 
                for name in file_names:
                    with open(name ,'rb') as f: 
                          loaded_obj = pickle.load(f)     
                          voice_files.append(loaded_obj)

                self.pr_dict[self.labels[iLabel]] = voice_files 


    def __len__(self):
        return len(self.pr_dict[self.labels[0]])

    def __getitem__(self, idx):      
        out_list = []
        
        if self.pr_dict["nbr_voices"][idx] == 4:
            for key, value in self.pr_dict.items():
              out_list.append(self.pr_dict[key][idx])
                              
            v0 = torch.tensor(out_list[0].T)
            v1 = torch.tensor(out_list[1].T)
            v2 = torch.tensor(out_list[2].T)
            v3 = torch.tensor(out_list[3].T)
            v_all = torch.tensor(out_list[4].T) 
            length = self.pr_dict["length"][idx]
            nbr_voices = self.pr_dict["nbr_voices"][idx]
            file_name = self.pr_dict["name"][idx]

            voices = torch.stack([v0, v1, v2, v3, v_all], dim=2)
            
            return (voices, length, nbr_voices, file_name)
        
        if self.pr_dict["nbr_voices"][idx] == 3:

            for key, value in self.pr_dict.items():
                if key != "voice_3":
                  out_list.append(self.pr_dict[key][idx]) 
            
            v0 = torch.tensor(out_list[0].T)
            v1 = torch.tensor(out_list[1].T)
            v2 = torch.tensor(out_list[2].T)
            v3 = torch.zeros(v2.shape)
            v_all = torch.tensor(out_list[3].T) 
            length = self.pr_dict["length"][idx]
            nbr_voices = self.pr_dict["nbr_voices"][idx]
            file_name = self.pr_dict["name"][idx]
            
            voices = torch.stack([v0, v1, v2, v3, v_all], dim=2)

            
            return (voices, length, nbr_voices, file_name)

In [ ]:
dataset = MusicDataset_polyphonic(PATH_TO_DATA)

loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False, num_workers=workers, drop_last=True)